# How to start

Before starting you must:
- Ensure that `scipp` and `mantid` are on your `PYTHONPATH`.
- Generate the `scippconfig.py` file using `make_config.py` located in same directory as this notebook. Refer to the `README.md` or `python make_config.py --help` for information.
- Install dependencies : `conda install fabio tifffile` (used for image handling)

For `scipp` and `mantid` follow instructions at: https://scipp.readthedocs.io/en/latest/getting-started/installation.html.

Converted to use scipp and notebook from [this repository](https://git.esss.dk/testbeamline/gp2/-/tree/stressexperiment).

For Table of Contents install Jupyter extensions then reload the notebook:
`conda install -c conda-forge jupyter_contrib_nbextensions`
`jupyter contrib nbextension install --user`
`jupyter nbextension enable toc2/main`

# Experimental Summary

This script has been developed to measure local strain ε defined as ε = ΔL/L0 in a FCC steel sample under elastic strain in a stress rig. Measured at V20, HZB, Berlin, September 2018 by Peter Kadletz.

λ = 2dsinθ, where 2θ = π (transmission), edges characterise the Bragg condition and hence λ = 2d. Therefore strain is easily computed from the wavelength measurement of of a Bragg edge directly, using un-loaded vs loaded experimental runs (and reference mesurements). The known Miller indices of the crystal structure (FCC) are used to predict the wavelength where the Bragg edges should exist, which is bound by the reachable wavelength extents for the instrument. This provides an approximate region to apply a fit.  A complement error function is used to fit each Bragg edge, and a refined centre location (λ) for the edge is used in the strain measurement. Because each bragg edge can be identified individually, one can determine anisotropic strain across the unit cell in the reachable crystallographic directions. In addition the image processing allows for spacial grouping so localised effects, such as those on unconstrained edges of the sample or in necking regions of the sample can be treated seperately. The plotted outputs in the script aim to capture this.





# Script setup

In [ ]:
try:
    import scipp
except ImportError as e:
    print("scipp is not available in the PYTHONPATH")
    raise e
    
try:
    import mantid
except ImportError as e:
    print("mantid is not available in the PYTHONPATH")
    raise e
    
try:
    import scippconfig
except ImportError as e:
    print("scippconfig is not available. Make sure you have generated it with `make_config.py`.")
    raise e

## Set input and output dirs

If your input directory has a different structure this is the cell to modify. 
Additionally the output directory can be renamed too.

In [ ]:
# Lets get everything set up
from multiprocessing.dummy import Pool as ThreadPool
import os
import sys

import scipp as sc
import numpy as np

import imaging, operations

from scipy import ndimage, signal

DATA_DIR_NAME = "data_GP2"
OUTPUT_DIR_NAME = "output"

data_dir = os.path.join(scippconfig.script_root, DATA_DIR_NAME)
output_dir = os.path.join(scippconfig.script_root, OUTPUT_DIR_NAME)
instrument_file = os.path.join(scippconfig.script_root, 'IDF', 'V20_Definition_GP2.xml')

tofs_path = os.path.join(data_dir, 'metadata', 'GP2_Stress_time_values.txt')
raw_data_dir = os.path.join(data_dir, "Stress_Experiments")

if not os.path.exists(data_dir):
    raise FileNotFoundError("The following data directory does not exist,"
                            f" check your make_config.py:\n{data_dir}")

## Reduction Options

In [ ]:
# Customisable Options:

# defining grouping of 2D detector pixels
grouping_number = 27
nx_target = grouping_number
ny_target = grouping_number

# Rebin regions for each of the 5 frames
# in the format of [bin-start, bin-end, bin width].
# used to crop each image, before stitching them together
frame_parameters = [(15450, 22942, 64),
                    (24800, 32052, 64),
                    (33791, 40084, 64),
                    (41763, 47457, 64),
                    (49315, 54500, 64),
                    (56500, 58360, 64)]

# Used to shift the cropped frames so that their bins overlap 
# before summing them together into a single frame
frame_shift_increments = [-6630, -2420, -2253, -2095, -1946, -1810]
frame_shift_increments = [float(i) for i in frame_shift_increments]  # Work around #1114

# Used to rebin the summed frame in order to
# cut off frames that contain no data
rebin_parameters = {"start": 8550, "stop": 26000, "num_bins": 110}

# Pulse references
pulse_number_reference = 1.0 / 770956
pulse_number_sample = 1.0 / 1280381
pulse_number_sample_elastic = 1.0 / 2416839
pulse_number_sample_plastic = 1.0 / 2614343

# units of transmission, all pixels with transmission higher masking threshold are masked
masking_threshold = 0.80

# Toggles outputting masked and sliced tiff stacks
output_tiff_stack = True

# Experiment Metadata
measurement_number = 11

# Reduction

## Load and scale

In [ ]:
# let's get the process started:

# Load X values from the TOF file
ds = sc.Dataset()
ds.coords["tof"] = sc.Variable(["tof"], unit=sc.units.us, values=imaging.read_x_values(tofs_path))
ds.coords["tof"] *= 1e3

def load_and_scale(folder_name, scale_factor):
    to_load = os.path.join(raw_data_dir, folder_name)
    variable = imaging.tiffs_to_variable(to_load)
    variable *= scale_factor
    return variable

ds["reference"] = load_and_scale(folder_name="1) R825 Open Beam", scale_factor=pulse_number_reference)
ds["sample"] = load_and_scale(folder_name="2) R825", scale_factor=pulse_number_sample)
ds["sample_elastic"] = load_and_scale(folder_name="3) R825 600 Mpa", scale_factor=pulse_number_sample_elastic)
ds["sample_plastic"] = load_and_scale(folder_name="4) R825 3500 um", scale_factor=pulse_number_sample_plastic)

## Add TOF and Spectra Coords

In [ ]:
# Adds a coordinate for the spectra and TOF
ds.coords["spectrum"] = sc.Variable(["spectrum"], values=np.arange(ds["sample"].shape[1]))
stitched = sc.Dataset(coords={"tof": sc.Variable(["tof"], unit=sc.units.us,
                              values=np.linspace(start=rebin_parameters["start"], stop=rebin_parameters["stop"],
                                                 num=rebin_parameters["num_bins"]))})

## Frame shifts and stitching

Pre-calculated frame shifts. This will be replaced with automatic WFM stitching in the future

In [ ]:
# TODO use Neil's wfmess, or similar to avoid hard-coding the edges. This approach is not very nice.
frame_shifts = [sum(frame_shift_increments[:i + 1]) for i in range(len(frame_shift_increments))]

def stitch_data(variable_to_stitch):
    print(f"Stitching: {variable_to_stitch}")
    stitched[variable_to_stitch] = imaging.stitch(ds[variable_to_stitch],
                                                  frame_parameters=frame_parameters,
                                                  frame_shifts=frame_shifts,
                                                  rebin_parameters=rebin_parameters)

for var_name in ds.keys():
    stitch_data(var_name)

## Masking
### Integration step

Integrate counts for each spectra by summing over TOF

In [ ]:
integrated = sc.sum(stitched, 'tof')

# Pull out reference from out dataset to avoid checking for it in loops
reference = stitched["reference"].copy()
integrated_reference = integrated["reference"].copy()

del stitched["reference"]
del integrated["reference"]


### Transmission Masking

Divides the integrated sample counts with an open beam reference. Any values > masking threshold
will be masked. The adj pixels step checks for random pixels which were left unmasked or masked
with all their neighbours having the same mask value. These are forced to True or false depending on
their neighbour value.

In [ ]:
num_spectra = stitched.coords["spectrum"].shape[0]
bank_width = int(np.sqrt(num_spectra))

def mask_non_sample_regions(integrated_dataset, stitched_dataset, var_name):
    integrated_spectra = integrated_dataset[var_name]

    # This should use sc.greater once #1178 is completed
    spectra_masks = np.greater(integrated_spectra.values, masking_threshold)
    spectra_masks = sc.Variable(["spectrum"], values=spectra_masks)

    # Some pixels may be noisy / not strong enough signal so they erranously
    # become unmasked / masked from the above operation. This step checks if *all*
    # the surrounding pixels are True or False then sets the center pixel to
    # that value if they are, removing "salt and pepper" noise
    spectra_masks = sc.reshape(spectra_masks, dims=["y", "x"], shape=(bank_width, bank_width))

    final_mask = operations.mask_from_adj_pixels(spectra_masks)
    return sc.Variable(["spectrum"], values=final_mask.values.ravel())

# Calculate transmission value for all
integrated /= integrated_reference

# Store TOF coords back for the export to TIFF stack
tof_coords = stitched.coords["tof"]

for k in integrated.keys():
    print(f"Masking non-sample regions in {k}")
    stitched.masks[k] = mask_non_sample_regions(integrated, stitched, k)

    if output_tiff_stack:
        print(f"Exporting tiff stack for {k}")
        tiff_out_dir = os.path.join(output_dir, f"tiffs_tof_sum_{k}")
        imaging.export_tiff_stack(dataset=integrated, key=k,
                                  base_name=f"{k}_norm_sum",
                                  output_dir=tiff_out_dir,
                                  x_len=324, y_len=324,
                                  tof_values=tof_coords.values)
        # Print a newline to prevent the saving messages overlapping
        print()

## Normalization

In [ ]:
# Normalize by open beam
normalized = stitched / reference

# Replace special values nan and inf
replacement=sc.Variable(value=0.0, variance=0.0)
kwargs = {"nan" : replacement, "posinf" : replacement, "neginf" : replacement}
for k in normalized.keys():
    sc.nan_to_num(normalized[k].data, out=normalized[k].data, **kwargs)

### Output Intermediate

In [ ]:
if output_tiff_stack:
    tof_values = normalized.coords["tof"].values

    imaging.export_tiff_stack(dataset=normalized, key="sample", x_len=324, y_len=324,
                              base_name="initial_tof", output_dir=os.path.join(output_dir, "tiffs_tof_initial"),
                              tof_values=tof_values)
    imaging.export_tiff_stack(dataset=normalized, key="sample_elastic", x_len=324, y_len=324,
                              base_name="elastic_tof", output_dir=os.path.join(output_dir, "tiffs_tof_elastic"),
                              tof_values=tof_values)
    imaging.export_tiff_stack(dataset=normalized, key="sample_plastic", x_len=324, y_len=324,
                          base_name="elastic_tof", output_dir=os.path.join(output_dir, "tiffs_tof_plastic"),
                              tof_values=tof_values)

## Convert to wavelength

In [ ]:
# Load in IDF for positions of source/sample...etc
sc.compat.mantid.load_component_info(normalized, instrument_file)
wavelength = sc.neutron.convert(normalized, "tof", "wavelength")

# Position data ...etc. is dropped on conversion but is later required for plotting
sc.compat.mantid.load_component_info(wavelength, instrument_file)

## Apply median filter 

In [ ]:
# Precalculate rather than doing this every loop, this will be removed in the future
neighbour_spectra = operations.generate_neighbouring_spectra(num_spectra=num_spectra, bank_width=bank_width)

def median_filter(dataset, variable_key):
    # TODO rewrite without loops
    print(f"Running median of workspace {variable_key}")
    masks = dataset.masks[variable_key]

    original_data = dataset[variable_key]
    filtered_var = dataset[variable_key].copy()
    for spec_num in dataset.coords["spectrum"].values:
        # We don't have to check `isMonitor(spec)` since coords will only return non-monitor spectra
        if masks.values[spec_num]:
            continue

        adj_spectra = neighbour_spectra[spec_num] + [spec_num]  # Including self
        neighbour_tof = np.array([original_data["spectrum", k].values for k in adj_spectra if not masks.values[k]])
        #y = np.median(neighbour_tof, axis=0) # takes median of each time bin across the 9 workspaces

        # median decision list counts the number of median occurances of each spectrum
        median_decision = np.zeros(len(neighbour_tof))

        # stepping through the time bins and looking for median
        for c in range(len(neighbour_tof[0])):
            # values of the same time bins across all neighbouring spectra
            neighbour_bin_values = neighbour_tof[:,c]

            # check if values are identical, e.g. all elements are zero
            if np.all(neighbour_bin_values == neighbour_bin_values[0]):
                continue

            # sorting those values of the same time bin and return as indices of array (argsort).
            neighbour_ind_sort = np.argsort(neighbour_bin_values)

            # for array with odd number of elements, sort and take centre value
            if len(neighbour_bin_values) % 2:
                median_decision[ neighbour_ind_sort[len(neighbour_ind_sort)//2] ] += 1
            # for array with even number of elements, sort and take both high and low centre value
            elif len(neighbour_bin_values) % 2 == 0:
                median_decision[ neighbour_ind_sort[(len(neighbour_ind_sort)-1) // 2] ] += 1 # low median
                median_decision[ neighbour_ind_sort[len(neighbour_ind_sort) // 2] ] += 1 # high median

        # choosing spectrum according to median_decision list
        # TODO argmax returns first maximum value. In case of many maximum values make average of spectra, or find other reasonable criteria.
        y = neighbour_tof[np.argmax(median_decision)]

        # TODO ws.setE. Uncertanties not delt with
        filtered_var["spectrum", spec_num].values = y

    return filtered_var
    
# TODO Peter has a new method for doing this which is probably simplier and would make dealing with uncertainies easier. Essentially avoid splitting by TOF bin.
# TODO Median filter is somewhat experimental. Peter has suggested that not grouping and then an average filter would also be a valid procesing route.
for k in wavelength.keys():
    wavelength[k] = median_filter(dataset=wavelength, variable_key=k)
    # TODO export_to_tiff_stack for each

## Rebin to common bin width

As each detector has a different relative position to the sample we
need to rebin to a common set of wavelength bins for all

In [ ]:
wav_x_vals = wavelength["sample"].coords["wavelength"]
start_wave = sc.min(wav_x_vals)
stop_wave = sc.max(wav_x_vals)

wavelength_coords = sc.Variable(["wavelength"], values=np.linspace(start_wave.value, stop_wave.value,
                                                                   num=rebin_parameters["num_bins"]))
rebinned_wavelength = sc.rebin(wavelength, "wavelength", wavelength_coords)


group_size = (324.0  / nx_target) * (324.0 / ny_target)
factor = group_size
rebinned_wavelength["sample"] *= factor

## Sample - Sum Spectra

Performs a weighted sum for all spectra in the sample 

In [ ]:
def weighted_sum(variable : sc.Variable, dim):
    variances = sc.Variable(dims=variable.dims, values=variable.variances)
    d = sc.Dataset()
    d['data'] = variable / variances
    d['norm'] = sc.reciprocal(variances)

    d.masks['zero_error'] = sc.equal(variances, 0.0 * variances.unit)
    
    d = sc.sum(d, dim)
    sum = d['data']/d['norm']
    return sum

# If we are not interested in strain regions for the unloaded sample, we can just combine all spectra to improve statistics.
# maybe another masking before summation, with tougher threshold to exclude boderline pixels.
sample_final = weighted_sum(rebinned_wavelength["sample"], "spectrum")

## Sample elastic - Group detectors

Group detectors instead of summing all, since the elastic deformation could
happen at any point we want a balance between signal-noise ratio and 
resolution.

In [ ]:
rebinned_wavelength.coords["spectrum_mapping"] = imaging.make_detector_groups(bank_width, bank_width,
                                                                              nx_target, ny_target)

# Group detectors for the sample_elastic / sample_plastic
grouped = sc.groupby(rebinned_wavelength, "spectrum_mapping").sum("spectrum")
grouped_sample_elastic = grouped["sample_elastic"]

## Pre-calculate detector positions

Pre-calculate average detector positions for plotting later

In [ ]:
histogram_matrices = [sample_final, grouped_sample_elastic]
template_histogram = histogram_matrices[1]

# Average out detector positions
positions_dataset = sc.Dataset()
positions_dataset["position"] = sc.Variable(rebinned_wavelength.coords["position"])
positions_dataset.coords["spectrum_mapping"] = rebinned_wavelength.coords["spectrum_mapping"]
grouped_positions = sc.groupby(positions_dataset, "spectrum_mapping").mean("spectrum")

plots = []
for spec_idx in template_histogram.coords["spectrum_mapping"].values:
    pos = grouped_positions["position"]["spectrum_mapping", int(spec_idx)]
    # Append spectrum index and position
    plots.append((spec_idx,pos))

# We cant access position using (dim, I), since the vector 3d has not dim currently
Y_key = 1
X_key = 0

# We must use round to prevent floating points errors during sorting like -0.05 > 0.05
# Sort by y then by x across image plots contain tuples of (spec_index to position), sort by position y, x
plots.sort(key=lambda t: (round(-t[1].value[Y_key], 10), round(t[1].value[X_key], 10)))

# Fitting
## Calculate expected Bragg edge positions 

In [ ]:
# Bragg edge position roughly, in Angstrom
FCC_a = 3.5 # Angstrom, taken from COD entry 9008469

# These miller indices for the given unit cell yield bragg edges between the maximum and minimum wavelength range. 
#  TODO. should probably calculate which are visible from larger set.
indices_FCC = [(1,1,1),(2,0,0),(2,2,0),(3,1,1)]

def create_Braggedge_list(lattice_constant, miller_indices):
    '''
    :param miller-indices: like [(1,1,0),(2,0,0),...]
    :type miller-indices: list of tuples
    '''
    coords = [str((h, k, l)) for h, k, l in miller_indices]    
    interplanar_distances = [2.*lattice_constant/np.sqrt(h**2+k**2+l**2) for h, k, l in miller_indices]

    d = sc.DataArray(sc.Variable(dims=["bragg-edge"], values=np.array(interplanar_distances),
                                  unit=sc.units.angstrom))
    d.coords["miller-indices"] = sc.Variable(dims=["bragg-edge"], values=coords)
    return d


Bragg_edges_FCC = create_Braggedge_list(FCC_a, indices_FCC)

## Bragg-edge fitting

Fit Bragg-edge peaks for the sample initially. Then take this value to find the position within a window
for the elastic sample.

In [ ]:
def convert_data_array(data_array, dim):
    # We can't use data_array_to_ws_2d in the compat layer as we need to
    # transpose X and Y, else Mantid has the wrong dims
    x_vals = np.array(data_array.coords["wavelength"].values)

    ws = sc.compat.mantid.to_mantid(data=data_array,dim=dim,
                                          instrument_file=instrument_file)
    return ws

def Bragg_edge_position(xpos_Bragg_edge,x_min_sides,x_max_sides,x_size,y_size,plots):
    '''Takes limits around a roughly known Bragg edge and fits the Bragg edge to obtain
    its exact position. Needs the size (x_size, y_size) and the workspaces containing histogram arrays/matrices.
    x_min_side and x_max_side are arbitrary values and are calculated:
    bragg_edge_pos * x_min_side or bragg_edge_pos * x_max_side.
    '''

    fit_list = []
    spectrum_list_fitted_sample = []
    spectrum_list_fitted_sample_elastic = []
    elastic_ws = grouped_sample_elastic
    sample_ws = sample_final

    # loop for each Bragg edge in the list, find edge positions and save spectrums in lists
    for edge_index in range(xpos_Bragg_edge.shape[0]):
        fit_list.append([])
        spectrum_list_fitted_sample.append([])
        spectrum_list_fitted_sample_elastic.append([])

        bragg_edge = xpos_Bragg_edge.coords["miller-indices"]["bragg-edge", edge_index]

        xpos_guess = xpos_Bragg_edge["bragg-edge", edge_index].value
        x_min_fit = xpos_guess - xpos_guess * abs(x_min_sides[edge_index])
        x_max_fit = xpos_guess + xpos_guess * abs(x_max_sides[edge_index])

        print("Now fitting Bragg edge {} at {:.3f} A (between {:.3f} A and {:.3f} A) across image groups"
              .format(bragg_edge, xpos_guess, x_min_fit, x_max_fit))

        # if the full inital sample was taken and no grouping was done
        #Fitting the masked sample
        params_s, diff_s = sc.compat.mantid.fit(sample_ws, mantid_args={
                                      'Function':f'name=LinearBackground,A0={230},A1={-4};name=UserFunction,Formula=h*erf(a*(x-x0)),h={16},a={-11},x0={xpos_guess}',
                                      'StartX':x_min_fit, 'EndX':x_max_fit})

        v_and_var_s=[params_s.data['parameter', i] for i in range(params_s['parameter',:].shape[0])] 
        params = dict(zip(params_s.coords["parameter"].values, v_and_var_s))
        d_sample = params["f1.x0"] / 2.0 # See fit table definition for extract x0
       
        plot_cell_idx = 0 
        for row in range(y_size):
            for col in range(x_size):
                spectrum_idx = int(plots[plot_cell_idx][0])
                
                # Fit Bragg edge using values from fit of unstrained sample
                elastic_function = f"name=LinearBackground,A0={params['f0.A0'].value},A1={params['f0.A1'].value};name=UserFunction,Formula=h*erf(a*(x-x0)),h={params['f1.h'].value},a={params['f1.a'].value},x0={params['f1.x0'].value}"
                params_e, diff_e = sc.compat.mantid.fit(elastic_ws['spectrum_mapping', spectrum_idx], mantid_args={'Function':elastic_function,
                                                   'StartX':x_min_fit, 'EndX':x_max_fit})
                
                v_and_var_e=[params_e.data['parameter', i] for i in range(params_e['parameter',:].shape[0])] 
                elastic_params = dict(zip(params_e.coords["parameter"].values, v_and_var_e))
                d_sample_elastic = elastic_params["f1.x0"] / 2.0 # See fit table definition for extract x0
                lattice_strain = d_sample_elastic - d_sample

                # define successful fitting
                success = (params_s.attrs["status"].value == "success") and (params_e.attrs["status"].value == "success")

                # fitted values STORED in list
                fit_list[edge_index].append((spectrum_idx, (row, col), bragg_edge, d_sample, d_sample_elastic, lattice_strain, success))

                # workspace created from sample fit, workspace index 1 for fitted spectrum (index 0 for data, index 2 for difference curve)
                fitted_sample = diff_s['calculated'].values

                if params_s.attrs["status"].value == "success":
                    spectrum_list_fitted_sample[edge_index].append((diff_s.coords["wavelength"].values, fitted_sample))
                else:
                    spectrum_list_fitted_sample[edge_index].append((fitted_sample.coords["wavelength"].values,
                                                           np.zeros_like(fitted_sample)))
                    
                fitted_elastic = diff_e['calculated'].values
                # workspace created from fit of sample under elastic deformation
                if params_e.attrs["status"].value == "success":
                    spectrum_list_fitted_sample_elastic[edge_index].append((diff_e.coords["wavelength"].values, fitted_elastic))
                else:
                    spectrum_list_fitted_sample_elastic[edge_index].append((diff_e.coords["wavelength"].values,
                                                                                               np.zeros_like(fitted_elastic)))
                plot_cell_idx +=1

    return fit_list, spectrum_list_fitted_sample, spectrum_list_fitted_sample_elastic

fit_list, spectrum_list_fitted_sample, spectrum_list_fitted_sample_elastic = \
    Bragg_edge_position(xpos_Bragg_edge=Bragg_edges_FCC,
                                      x_min_sides=[0.05, 0.1, 0.1, 0.05],#[0.1, 0.1, 0.1, 0.05]
                                      x_max_sides=[0.1, 0.05, 0.1, 0.1],#[0.1, 0.1, 0.1, 0.1]
                                      x_size=nx_target,
                                      y_size=ny_target, 
                                      plots=plots)

# Save results
## Write out ASCII

In [ ]:
def write_ws_to_ascii(input_list, output_filename_stem, output_directory):
    '''
    Takes histogram from mantid workspace and writes an ASCII file.
    New filename will be the filename of the input workspace and
    a given extension appended.

    :param input_ws: input workspace from mantid
    :param output_directory: Where to write the new file
    :param output_extension: which extension should the ASCII file have

    :raises: Error if file exists
    '''

    output_filenames = []
    for i, li in enumerate(input_list):
        bragg_edge = li[0][2].value
        output_filename = "_{}_{:.3f}A_{}.txt".format(output_filename_stem, li[0][3].value, bragg_edge)
        if not isinstance(li, list):
            print("Input is not a list.")
            return
        whole_path = os.path.join(output_directory,output_filename)
        if os.path.isfile(whole_path):
            sys.stderr.write("ERROR: file {:s} already exists!\n".format(output_filename))
            return

        # TODO replace with np.savetxt
        with open(whole_path, 'w') as output_file:
            for line in li:
                for element in line:
                    if not element == line[-1]:
                        output_file.write("{}\t".format(element))
                    else:
                        output_file.write("{}\n".format(element))
        print("File {} was created.".format(output_filename))
        output_filenames.append(output_filename)
    return output_filenames

tbin_width = 64*2.5
output_filename_table_stem = '{:03d}_table_strain_analysis_{}xy_{}usbin_{}thresh_initintegr'.format(
                        measurement_number,grouping_number,tbin_width,masking_threshold)



write_ws_to_ascii(fit_list, output_filename_table_stem, output_dir)

## Generate single tile plot

In [ ]:
import matplotlib.pyplot as plt

def tileplot_colorcode(fit_list, nx_target, ny_target, outlier_threshold, output_filename_stem='tileplot_color'):
    # for each part of the fit list containing the fit values of each Bragg edge, create a color plot.
    output_filename_list = []
    for fit_be_list in fit_list:
        bragg_edge = fit_be_list[0][2].values  # the calculated Bragg edge TOF position
        d_spacing = fit_be_list[0][3] # the calculated Bragg edge TOF position
        output_filename = '{}_{:.3f}A_{}'.format(output_filename_stem, d_spacing.value, bragg_edge)
        # Make a 2D image of the strain values
        print(('Plotting color-coded tile plot of Bragg edge {}.'.format(bragg_edge)))
        fig, ax = plt.subplots()
        strains = np.zeros([ny_target, nx_target])
        plots_counter = 0
        for row in range(ny_target):
            for col in range(nx_target):
                if (fit_be_list[plots_counter][-1] and outlier_threshold[0] < fit_be_list[plots_counter][-2].value < outlier_threshold[1]):
                    strains[row, col] = fit_be_list[plots_counter][-2].value
                plots_counter += 1
        import matplotlib.colors as colors
        im = ax.imshow(strains, origin="upper", norm=colors.SymLogNorm(linthresh=1.0e-3), cmap="RdBu")
        cb = plt.colorbar(im)
        cb.set_label("${}$ Lattice strain $\\varepsilon$".format(bragg_edge))
        print(f'Saving {output_filename}.pdf and .png.')
        fig.savefig("{}.pdf".format(output_filename), bbox_inches="tight")
        fig.savefig("{}.png".format(output_filename), bbox_inches="tight")
        output_filename_list.append(output_filename)
    return output_filename_list



# i.e. -1e-2, 1e-2 means only -1% to 1% values of lattice strain are shown
outlier_threshold_color_plot = (-5e-2, 5e-2)

output_filename_tileplot_color ='{:03d}_tileplot_color_{}xy_{}usbin_{}thresh_{:.3f}-{:.3f}plotthresh_initintegr'.format(
                        measurement_number,grouping_number,tbin_width,masking_threshold,
                        outlier_threshold_color_plot[0], outlier_threshold_color_plot[1])

fignames_col = tileplot_colorcode(fit_list=fit_list, nx_target=nx_target, ny_target=ny_target,
                                  outlier_threshold=outlier_threshold_color_plot,
                                  output_filename_stem=os.path.join(output_dir, output_filename_tileplot_color))

## Generate tile plots for single fig.

In [ ]:
def edges_to_centers(x):
    """
    Convert array edges to centers
    """
    return 0.5 * (x[1:] + x[:-1])

def tileplot(x_min_plot,x_max_plot,y_min_plot,y_max_plot,x_size,y_size,
             histogram_matrices,fitted_data, plots, fit_list, output_filename='tileplot_curves'):

    fig, ax = plt.subplots(x_size, y_size, figsize=(12, 12))

    labels = ["Without load", "Elastic deformation"]
    markers = ["o", "s"]
    # Plot some fake data so that we can have larger markers in the legend
    ax[0][0].plot([0, 1], [0, 1], ls="None", marker=markers[0], markersize=4, label=labels[0], color="C0")
    ax[0][0].plot([0, 1], [0, 1], ls="None", marker=markers[1], markersize=4, label=labels[1], color="C1")

    # All bins are identical in a dataset
    x0_nostrain = edges_to_centers(histogram_matrices[0].coords["wavelength"].values)

    # x and y values of inital state, samples without strain
    y0_nostrain = histogram_matrices[0].values

    plots_counter = 0
    for row in range(y_size):
        for col in range(x_size):
            # for running workspace index
            ws_index = plots[plots_counter][0]

            # if the whole inital-sample-state is just one spectrum, always plot that.
            # Plot no-strain curve, inital state of sample
            ax[row][col].plot(x0_nostrain, y0_nostrain, ls="None", marker= "o", markersize=1)

            # Plot curves of other states given in histogram_matrices
            for k in range(len(histogram_matrices)):
                    x0 = edges_to_centers(histogram_matrices[1].coords["wavelength"].values)
                    y0 = histogram_matrices[1]["spectrum_mapping", int(ws_index)].values
                    ax[row][col].plot(x0, y0, ls="None", marker=markers[k], markersize=1)

            # plotting the fitted spectra:
            x1 = edges_to_centers(fitted_data[0][plots_counter][0])
            x2 = edges_to_centers(fitted_data[1][plots_counter][0])
            ax[row][col].plot(x1, fitted_data[0][plots_counter][1], label="{} fit".format(labels[0]), lw=1, zorder=3)
            ax[row][col].plot(x2, fitted_data[1][plots_counter][1], label="{} fit".format(labels[1]), lw=1, zorder=3)

            # Set axis limits
            ax[row][col].set_xlim([x_min_plot, x_max_plot])
            ax[row][col].set_ylim([y_min_plot, y_max_plot])
            if col == x_size - 1:
                ax[row][col].yaxis.tick_right()
            else:
                ax[row][col].set_yticklabels([])
            if row == 0:
                ax[row][col].xaxis.tick_top()
            else:
                ax[row][col].set_xticklabels([])
            ax[row][col].tick_params(axis="x", direction="in", bottom=True, top=True)
            ax[row][col].tick_params(axis="y", direction="in", left=True, right=True)

            if row == y_size - 1:
                ax[row][col].text(0.0, -0.1, "{:.1f}".format(0.1 * col),
                                ha='center',va='top', transform=ax[row][col].transAxes)
            if col == 0:
                ax[row][col].text(-0.1, 0.0, "{:.1f}".format(0.1*(y_size - row - 1)),
                                ha='right',va='center', transform=ax[row][col].transAxes)

            if fit_list[plots_counter][-1]: # usual sign for lattice strain is \\varepsilon
                ax[row][col].text(0.1, 0.8, "${:.3f}$".format(fit_list[plots_counter][-2].value),
                                ha='left',va='top', transform=ax[row][col].transAxes, fontsize=4)
            plots_counter += 1

    # Last bin edge for xy coordinates
    ax[-1][-1].text(1.0, -0.1, "{:.1f}".format(0.1 * x_size),
                    ha='center',va='top', transform=ax[-1][-1].transAxes)
    ax[0][0].text(-0.1, 1.0, "{:.1f}".format(0.1 * y_size),
                  ha='right',va='center', transform=ax[0][0].transAxes)
    # Display only one legend
    ax[0][0].legend(loc=(0, 1.8), ncol=4)
    fig.text(0.5, 0.905, "Wavelength $[\mathrm{\AA}]$")
    fig.text(0.94, 0.5, "Counts", rotation=90, ha='center',va='center')
    fig.text(0.5, 0.075, "X position [m]")
    fig.text(0.09, 0.5, "Y position [m]", rotation=90, ha='center',va='center')
    # Remove white space between subplots
    fig.subplots_adjust(wspace=0.0, hspace=0.0)
    print(('Saving {}.pdf.').format(output_filename))
    fig.savefig("{}.pdf".format(output_filename), bbox_inches="tight")
    fig.savefig("{}.png".format(output_filename), bbox_inches="tight")
    return output_filename

output_filename_tileplot = '{:03d}_tileplot_curves_{}xy_{}usbin_{}thresh_initintegr'\
    .format(measurement_number,grouping_number,tbin_width,
            masking_threshold)

# Plotting: y_min_plot=200,y_max_plot=270 for 18x18, y_min_plot=90,y_max_plot=120 for 27x27.
plotting_win = [(i*(1-.135), i*(1+.135)) for i in Bragg_edges_FCC.values]
for i in range(Bragg_edges_FCC.values.shape[0]):
    angstrom_val = Bragg_edges_FCC["bragg-edge", i].value / 2.
    lattice_indicies = Bragg_edges_FCC.coords["miller-indices"]["bragg-edge", i].value
    output_filename_tileplot_new = '{}_{:.3f}A_({})'.format(output_filename_tileplot, angstrom_val, lattice_indicies)

    win = plotting_win[i]
    fignames_curve = tileplot(
        x_min_plot=win[0], x_max_plot=win[1],
        y_min_plot=65610./(nx_target*ny_target), y_max_plot=87480./(nx_target*ny_target), # remove 0.9 later
        x_size=nx_target, y_size=ny_target,
        histogram_matrices=histogram_matrices,
        fitted_data=[spectrum_list_fitted_sample[i], spectrum_list_fitted_sample_elastic[i]],
        plots=plots, fit_list=fit_list[i], output_filename=os.path.join(output_dir, output_filename_tileplot_new))